In [282]:
"""
Скрипт для составления списка общежитий
"""

'\nСкрипт для составления списка общежитий\n'

In [283]:
import pandas as pd
import openpyxl
import time
from openpyxl.utils.dataframe import dataframe_to_rows

In [284]:
ab_df = pd.read_excel('data/Абитуриенты.xlsx',sheet_name='Абитуриенты',skiprows=8,dtype={'ИНН':str,'Серия УЛ':str,'Номер УЛ':str})
dop_df = pd.read_excel('data/доп статусы.xlsx',skiprows=3)

In [285]:
ab_df.head()

,№,ФИО,Пол,Дата рождения,Тип УЛ,Серия УЛ,Номер УЛ,Дата выдачи УЛ,Кем выдано УЛ,ИНН,...,Статус результатов ЕГЭ,Баллы за индивидуальные достижения,Сумма баллов за индивидуальные достижения,"Баллы за индивидуальные достижения, учитываемые как преимущество","Сумма баллов за индивидуальные достижения, учитываемые как преимущество",Организация-заказчик,Целевая контрактная подготовка,Uid ЕПГУ,Договор заключен,Договор оплачен
0,1,Абросова Анастасия Витальевна,Ж,08.07.2007,Паспорт РФ,8112,927500,04.08.2021,МВД ПО РЕСПУБЛИКЕ БУРЯТИЯ,030620313718,...,NaN,NaN,0.0,NaN,0.0,NaN,нет,NaN,нет,нет
1,2,Абсаматов Вадим Чимид-Цыренович,М,14.12.2007,Паспорт РФ,8120,900692,23.12.2021,МВД ПО РЕСП. БУРЯТИЯ,NaN,...,NaN,NaN,0.0,NaN,0.0,NaN,нет,NaN,нет,нет
2,3,Авдеев Сергей Александрович,М,09.10.2006,Паспорт РФ,8120,896798,23.12.2020,МВД по РЕСПУБЛИКЕ БУРЯТИЯ,NaN,...,NaN,NaN,0.0,NaN,0.0,NaN,нет,NaN,нет,нет
3,4,Авдеев Сергей Александрович,М,09.10.2006,Паспорт РФ,8120,896798,23.12.2020,МВД по РЕСПУБЛИКЕ БУРЯТИЯ,NaN,...,NaN,NaN,0.0,NaN,0.0,NaN,нет,NaN,нет,нет
4,5,Аверина Вероника Евгеньевна,Ж,03.12.2007,Паспорт РФ,8121,951540,17.12.2021,МВД ПО РЕСП. БУРЯТИЯ,NaN,...,NaN,NaN,0.0,NaN,0.0,NaN,нет,NaN,нет,нет


In [286]:
dop_df = dop_df[['Абитуриент','Доп. статус']]

In [287]:
ab_df = ab_df[ab_df['Состояние выбран. конкурса'] == 'Зачислен'] # оставялем только зачисленных
ab_df = ab_df[ab_df['Вид возмещения затрат'] == 'бюджет'] #отбираем только бюджетников

In [288]:
replace_dct = {'Отделение железнодорожного транспорта':'ЖД','Отделение металлообработки':'МО',
               'Отделение энергоснабжения':'ЭО','Хоринский филиал':'ХФ',}

In [289]:
ab_df['Формирующее подр.'] = ab_df['Формирующее подр.'].replace(replace_dct)

In [290]:
unit_lst = ab_df['Формирующее подр.'].unique()

In [291]:
dct_unit = {key:None for key in unit_lst}

In [292]:
ab_df.shape

(496, 108)

In [293]:
ab_df = ab_df[ab_df['Нуждается в общежитии']=='да']

In [294]:
ab_df.shape

(215, 108)

In [295]:
ab_df = ab_df[['ФИО','Контактный телефон','Мобильный телефон','Адрес регистрации','Формирующее подр.','Ср. балл док-та об образовании']]

In [296]:
ab_df

,ФИО,Контактный телефон,Мобильный телефон,Адрес регистрации,Формирующее подр.,Ср. балл док-та об образовании
6,Авсеевич Радмила Станиславовна,89516315807,NaN,"671178, Россия, с. Средний Убукун (03), Респ. ...",МО,3.350
10,Агеев Роман Алексеевич,89503889501,NaN,"671404, Россия, п. Майла (03), Респ. Бурятия, ...",МО,3.583
16,Алексеев Александр Викторович,89146324991,NaN,"671820, Россия, пгт. Наушки (03), Респ. Буряти...",ЖД,3.950
19,Алексеенко Григорий Эдуардович,89834233431,NaN,"671336, Россия, пгт. Онохой (03), Респ. Буряти...",МО,3.316
21,Алиева Дарья Борисовна,NaN,8.951631e+10,"671220, Россия, рп. Танхой (03), Респ. Бурятия...",ЭО,3.895
...,...,...,...,...,...,...
1208,Шангин Алексей Сергеевич,89025961998,NaN,"673055, Россия, с. Усть-Урлук (75), край Забай...",ЭО,3.350
1210,Шардин Денис Владимирович,89834272167,NaN,"671900, Россия, с. Джида (03), Респ. Бурятия, ...",МО,3.762
1213,Шимитов Баир Арсаланович,89369999332,NaN,"687000, Россия, пгт. Агинское (75), край Забай...",ЖД,3.700
1217,Ширинкина Елена Андреевна,89503882862,NaN,"671120, Россия, с. Нижний Саянтуй (03), Респ. ...",МО,3.750


In [297]:
merge_df = ab_df.merge(dop_df,left_on='ФИО',right_on='Абитуриент',how='inner')

In [298]:
merge_df.drop(columns=['Абитуриент'],inplace=True)

In [299]:
for name in unit_lst:
    temp_df = merge_df[merge_df['Формирующее подр.']== name]
    temp_df.sort_values(by='ФИО',inplace=True)
    dct_unit[name] = temp_df
    
    

C:\Users\1\AppData\Local\Temp\ipykernel_19544\1813645810.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_df.sort_values(by='ФИО',inplace=True)


In [300]:
t = time.localtime()
current_time = time.strftime('%H_%M_%S', t)

In [301]:
wb = openpyxl.Workbook()
# Переименовываем лист
sheet = wb['Sheet']
sheet.title = 'ЖД'
wb.create_sheet('МО',index=1)
wb.create_sheet('ЭО',index=2)
wb.create_sheet('ХФ',index=3)

<Worksheet "ХФ">

In [302]:
for name,df in dct_unit.items():    
    for r in dataframe_to_rows(df,index =False,header=True):
        if len(r) != 1:
            wb[name].append(r)

In [303]:
for name in unit_lst:
    wb[name].column_dimensions['A'].width =40
    wb[name].column_dimensions['B'].width =10
    wb[name].column_dimensions['C'].width =20
    wb[name].column_dimensions['D'].width =90


In [304]:
# Сохраняем итоговый файл
wb.save(f'data/Список на заселение со статусами от {current_time}.xlsx')

In [305]:
#pers_df = pers_df[['ФИО','Контактный телефон','Мобильный телефон','Адрес регистрации']] # получаем маленький датафрейм